In [1]:
!pip install imutils
!pip install pytube --upgrade
!pip install super_gradients
!pip install roboflow

%pip install clearml

In [2]:
import os
from super_gradients.training import Trainer
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train , coco_detection_yolo_format_val
from IPython.display import clear_output
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
from super_gradients.training import models

The console stream is logged into /root/sg_logs/console.log


[2024-05-12 20:23:44] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2024-05-12 20:23:53] INFO - utils.py - NumExpr defaulting to 2 threads.


In [3]:
CHECKPOINT_DIR = 'checkpoints2';
trainer = Trainer(experiment_name = 'human_in_thermal_run2' , ckpt_root_dir = CHECKPOINT_DIR)

In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="8alt1kWsWIlo930A0quA")
project = rf.workspace("chirag-otenn").project("human-detection-in-thermal")
version = project.version(1)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


In [5]:

%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=0BEQ9KSY152LX8NWYP2N
%env CLEARML_API_SECRET_KEY=JP9MPH219us6fWfecSbjvRD6jVGaXkk7odJKXUclrkES8TjjBM

In [6]:
from clearml import Task
task = Task.init(project_name='Human_Detection_Thermal_Images', task_name='Yolo_NAS_N')

ClearML Task: created new task id=52c5f34d7d0945d892c318af855dee25
ClearML results page: https://app.clear.ml/projects/dfc7076fbb8a4f4eb41470c5f03bd6e0/experiments/52c5f34d7d0945d892c318af855dee25/output/log


/usr/local/lib/python3.10/dist-packages/clearml/utilities/process/mp.py:622: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



In [7]:
dataset_params = {
    'data_dir':'/content/Human-Detection-in-Thermal-1',
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes':['Human']
}

In [8]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': str(dataset_params['data_dir']),
        'images_dir': str(dataset_params['train_images_dir']),
        'labels_dir': str(dataset_params['train_labels_dir']),
        'classes': str(dataset_params['classes']),
    },
    dataloader_params={
        'batch_size': 16,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

clear_output()


In [9]:
model = models.get('yolo_nas_s', num_classes=len(dataset_params['classes']),
                   pretrained_weights = "coco"
                   )

Indexing dataset annotations: 100%|██████████| 74/74 [00:00<00:00, 285.75it/s]
[2024-05-12 20:25:17] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco dataset, published under the following terms: https://cocodataset.org/#termsofuse
[2024-05-12 20:25:17] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.


2024-05-12 20:25:18,780 - clearml.model - INFO - Selected model id: 5c7dfded0e4345ba8b84046c08d9946e


[2024-05-12 20:25:21] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_s


In [10]:
train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": 16,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

In [11]:
trainer.train(model=model,
              training_params=train_params,
              train_loader=train_data,
              valid_loader=val_data)

[2024-05-12 20:25:24] INFO - sg_trainer.py - Starting a new run with `run_id=RUN_20240512_202524_167757`
[2024-05-12 20:25:24] INFO - sg_trainer.py - Checkpoints directory: checkpoints2/human_in_thermal_run2/RUN_20240512_202524_167757
[2024-05-12 20:25:24] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}
/usr/local/lib/python3.10/dist-packages/super_gradients/common/registry/registry.py:72: DeprecationWarning:

Object name `linear_epoch_step` is now deprecated. Please replace it with `LinearEpochLRWarmup`.

/usr/local/lib/python3.10/dist-packages/super_gradients/training/utils/optimizer_utils.py:108: DeprecationWarning:

initialize_param_groups and update_param_groups usages are deprecated since 3.4.0, will be removed in 3.5.0 and have no effect. 
 Assign different learning rates by passing a mapping of layer name prefixes to lr values through initial_lr training hyperparameter (i.e initial_lr={'backbone': 0.01, 'default':0.1})

/usr/lib/python3.10

The console stream is now moved to checkpoints2/human_in_thermal_run2/RUN_20240512_202524_167757/console_May12_20_25_24.txt


[2024-05-12 20:25:33] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               1          (1 available on the machine)
    - Full dataset size:            1824       (len(train_set))
    - Batch size per GPU:           16         (batch_size)
    - Batch Accumulate:             1          (batch_accumulate)
    - Total batch size:             16         (num_gpus * batch_size)
    - Effective Batch size:         16         (num_gpus * batch_size * batch_accumulate)
    - Iterations per epoch:         114        (len(train_loader))
    - Gradient updates per epoch:   114        (len(train_loader) / batch_accumulate)
    - Model: YoloNAS_S  (19.02M parameters, 19.02M optimized)
    - Learning Rates and Weight Decays:
      - default: (19.02M parameters). LR: 0.0005 (19.02M parameters) WD: 0.0, (42.13K parameters), WD: 0.0001, (18.98M parameters)

[2024-05-12 20:25:33] INFO - sg_trainer.py - Started training for 1

SUMMARY OF EPOCH 0
├── Train
│   ├── Ppyoloeloss/loss_cls = 1.7537
│   ├── Ppyoloeloss/loss_iou = 1.0381
│   ├── Ppyoloeloss/loss_dfl = 0.9281
│   └── Ppyoloeloss/loss = 3.72
└── Validation
    ├── Ppyoloeloss/loss_cls = 1.9637
    ├── Ppyoloeloss/loss_iou = 0.8127
    ├── Ppyoloeloss/loss_dfl = 0.7173
    ├── Ppyoloeloss/loss = 3.4937
    ├── Precision@0.50 = 0.0
    ├── Recall@0.50 = 0.0
    ├── Map@0.50 = 0.0033
    ├── F1@0.50 = 0.0
    └── Best_score_threshold = 0.04



Validating epoch 1: 100%|██████████| 9/9 [00:04<00:00,  1.92it/s]
[2024-05-12 20:31:47] INFO - base_sg_logger.py - Checkpoint saved in checkpoints2/human_in_thermal_run2/RUN_20240512_202524_167757/ckpt_best.pth
[2024-05-12 20:31:47] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.5116849541664124


...

Validating epoch 14: 100%|██████████| 9/9 [00:04<00:00,  1.93it/s]


SUMMARY OF EPOCH 14
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.9183
│   │   ├── Epoch N-1      = 0.9229 (↘ -0.0046)
│   │   └── Best until now = 0.9229 (↘ -0.0046)
│   ├── Ppyoloeloss/loss_iou = 0.543
│   │   ├── Epoch N-1      = 0.5429 (↗ 1e-04)
│   │   └── Best until now = 0.5429 (↗ 1e-04)
│   ├── Ppyoloeloss/loss_dfl = 0.5055
│   │   ├── Epoch N-1      = 0.5093 (↘ -0.0038)
│   │   └── Best until now = 0.5093 (↘ -0.0038)
│   └── Ppyoloeloss/loss = 1.9668
│       ├── Epoch N-1      = 1.9751 (↘ -0.0083)
│       └── Best until now = 1.9751 (↘ -0.0083)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.911
    │   ├── Epoch N-1      = 0.9013 (↗ 0.0097)
    │   └── Best until now = 0.9013 (↗ 0.0097)
    ├── Ppyoloeloss/loss_iou = 0.5248
    │   ├── Epoch N-1      = 0.5396 (↘ -0.0149)
    │   └── Best until now = 0.5304 (↘ -0.0057)
    ├── Ppyoloeloss/loss_dfl = 0.4775
    │   ├── Epoch N-1      = 0.4866 (↘ -0.0091)
    │   └── Best until now = 0.4839 (↘ -0.0064)
    ├── Ppyoloeloss/loss = 1.

Validating epoch 15: 100%|██████████| 9/9 [00:05<00:00,  1.64it/s]
[2024-05-12 21:16:22] INFO - base_sg_logger.py - Checkpoint saved in checkpoints2/human_in_thermal_run2/RUN_20240512_202524_167757/ckpt_best.pth
[2024-05-12 21:16:22] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.8324006795883179


SUMMARY OF EPOCH 15
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.9098
│   │   ├── Epoch N-1      = 0.9183 (↘ -0.0085)
│   │   └── Best until now = 0.9183 (↘ -0.0085)
│   ├── Ppyoloeloss/loss_iou = 0.5314
│   │   ├── Epoch N-1      = 0.543  (↘ -0.0116)
│   │   └── Best until now = 0.5429 (↘ -0.0115)
│   ├── Ppyoloeloss/loss_dfl = 0.4965
│   │   ├── Epoch N-1      = 0.5055 (↘ -0.009)
│   │   └── Best until now = 0.5055 (↘ -0.009)
│   └── Ppyoloeloss/loss = 1.9377
│       ├── Epoch N-1      = 1.9668 (↘ -0.0291)
│       └── Best until now = 1.9668 (↘ -0.0291)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.8749
    │   ├── Epoch N-1      = 0.911  (↘ -0.0361)
    │   └── Best until now = 0.9013 (↘ -0.0264)
    ├── Ppyoloeloss/loss_iou = 0.5295
    │   ├── Epoch N-1      = 0.5248 (↗ 0.0048)
    │   └── Best until now = 0.5248 (↗ 0.0048)
    ├── Ppyoloeloss/loss_dfl = 0.4798
    │   ├── Epoch N-1      = 0.4775 (↗ 0.0023)
    │   └── Best until now = 0.4775 (↗ 0.0023)
    ├── Ppyoloeloss/loss = 

[2024-05-12 21:16:44] INFO - sg_trainer.py - RUNNING ADDITIONAL TEST ON THE AVERAGED MODEL...
Validating epoch 16: 100%|██████████| 9/9 [00:07<00:00,  1.26it/s]


In [12]:
best_model = models.get('yolo_nas_s',
                        num_classes=len(dataset_params['classes']),
                        checkpoint_path="/content/checkpoints2/human_in_thermal_run2/RUN_20240512_202524_167757/ckpt_best.pth")

2024-05-12 21:19:43,315 - clearml.model - INFO - Selected model id: a0961033eb8046e8ac2d25359caeae68
2024-05-12 21:19:46,509 - clearml.model - WARNING - Connecting multiple input models with the same name: `ckpt_best`. This might result in the wrong model being used when executing remotely


In [13]:
# Validation
trainer.test(model=best_model,
            test_loader=val_data,
            test_metrics_list=DetectionMetrics_050(score_thres=0.1,
                                                   top_k_predictions=300,
                                                   num_cls=len(dataset_params['classes']),
                                                   normalize_targets=True,
                                                   post_prediction_callback=PPYoloEPostPredictionCallback(score_threshold=0.01,
                                                                                                          nms_top_k=1000,
                                                                                                          max_predictions=300,
                                                                                                          nms_threshold=0.7)
                                                  ))

Testing:  89%|████████▉ | 8/9 [00:05<00:00,  1.86it/s]

{'PPYoloELoss/loss_cls': 0.87493503,
 'PPYoloELoss/loss_iou': 0.52952635,
 'PPYoloELoss/loss_dfl': 0.4798191,
 'PPYoloELoss/loss': 1.8842803,
 'Precision@0.50': 0.042076416313648224,
 'Recall@0.50': 0.9684600830078125,
 'mAP@0.50': 0.8324006795883179,
 'F1@0.50': 0.08064889907836914,
 'Best_score_threshold': 0.47999998927116394}

Testing: 100%|██████████| 9/9 [00:06<00:00,  1.48it/s]


In [14]:
# Testing
trainer.test(model=best_model,
            test_loader=test_data,
            test_metrics_list=DetectionMetrics_050(score_thres=0.1,
                                                   top_k_predictions=300,
                                                   num_cls=len(dataset_params['classes']),
                                                   normalize_targets=True,
                                                   post_prediction_callback=PPYoloEPostPredictionCallback(score_threshold=0.01,
                                                                                                          nms_top_k=1000,
                                                                                                          max_predictions=300,
                                                                                                          nms_threshold=0.7)
                                                  ))

Testing:  80%|████████  | 4/5 [00:08<00:01,  1.38s/it]

{'PPYoloELoss/loss_cls': 0.8935924,
 'PPYoloELoss/loss_iou': 0.5336044,
 'PPYoloELoss/loss_dfl': 0.48779905,
 'PPYoloELoss/loss': 1.9149954,
 'Precision@0.50': 0.04032000154256821,
 'Recall@0.50': 0.9545454382896423,
 'mAP@0.50': 0.794327437877655,
 'F1@0.50': 0.07737182080745697,
 'Best_score_threshold': 0.4599999785423279}

In [15]:
import shutil
import os
import zipfile
directory_to_zip = '/content/checkpoints2/'
zip_file_path = '/content/YOLO_NAS_N.zip'
with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(directory_to_zip):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), directory_to_zip))

In [16]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [17]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [18]:
!pip install google-api-python-client
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload


auth.authenticate_user()

[2024-05-12 21:21:13] WARNING - _default.py - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2024-05-12 21:21:13] WARNING - _default.py - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2024-05-12 21:21:29] WARNING - _default.py - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


In [19]:
drive_service = build('drive', 'v3')

In [20]:
file_metadata = {'name': 'YOLO_NAS_N.zip'}
media = MediaFileUpload(zip_file_path, resumable=True)
file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
print(f"Uploaded '{zip_file_path}' to Google Drive with ID: {file.get('id')}")

Uploaded '/content/YOLO_NAS_N.zip' to Google Drive with ID: 16AOrStKa5G6NxAKD-6idOPBJ8X2q9JgK
